In [1]:
import pandas as pd
import re

In [2]:
data = pd.read_csv("./data/spotify_songs.csv")

In [3]:
print(data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18454 entries, 0 to 18453
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   track_id                  18454 non-null  object 
 1   track_name                18454 non-null  object 
 2   track_artist              18454 non-null  object 
 3   lyrics                    18194 non-null  object 
 4   track_popularity          18454 non-null  int64  
 5   track_album_id            18454 non-null  object 
 6   track_album_name          18454 non-null  object 
 7   track_album_release_date  18454 non-null  object 
 8   playlist_name             18454 non-null  object 
 9   playlist_id               18454 non-null  object 
 10  playlist_genre            18454 non-null  object 
 11  playlist_subgenre         18454 non-null  object 
 12  danceability              18454 non-null  float64
 13  energy                    18454 non-null  float64
 14  key   

In [4]:
def clean_text(lyric):
     return  re.sub(r'[^a-zA-Z\s]', '', lyric)

In [5]:
data = data[data['language'] == 'en']
data['lyrics'] = data['lyrics'].apply(clean_text)
data['track_name'] = data['track_name'].apply(clean_text)
data['track_album_name'] = data['track_album_name'].apply(clean_text)

In [6]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import re

C:\Users\rebec\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
tokenizer = AutoTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
model = AutoModelForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

tokenizer_config.json: 100%|██████████| 39.0/39.0 [00:00<00:00, 39.1kB/s]
C:\Users\rebec\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\rebec\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  wa

In [8]:
# Verifica si CUDA está disponible
if torch.cuda.is_available():
    model = model.to('cuda')
else: 
    raise Exception ("Must have CUDA available")

In [9]:
def sentiment_score(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    
    # Mueve los tensores de entrada al dispositivo correcto
    inputs = {k: v.to('cuda') for k, v in inputs.items()} if torch.cuda.is_available() else inputs

    with torch.no_grad():
        outputs = model(**inputs)

    scores = outputs[0][0].softmax(0)
    scores = scores.detach().cpu().numpy()  # Mueve el resultado a la CPU para convertirlo a numpy
    max_score = scores.argmax()
    return max_score


In [10]:
def process_in_batches(data, column, batch_size=32):
    num_batches = len(data) // batch_size + (0 if len(data) % batch_size == 0 else 1)
    results = []

    for i in range(num_batches):
        batch = data[column][i * batch_size:(i + 1) * batch_size]
        batch_result = batch.apply(sentiment_score)
        results.extend(batch_result)
    
    return results

In [11]:
data['lyrics_sentiment'] = process_in_batches(data, 'lyrics', batch_size=32)

In [12]:
data['album_name_sentiment'] = process_in_batches(data, 'track_album_name', batch_size=32)

In [13]:
data['track_name_sentiment'] = process_in_batches(data, 'track_name', batch_size=32)

In [14]:
data.to_csv("spotify_songs_processed.csv", index=False)